In [1]:
import pandas as pd
import random

In [2]:
from aizynthfinder.chem import Molecule, Reaction, MoleculeException
from aizynthfinder.training.utils import (
    Config,
    create_reactants_molecules,
    reverse_template,
    reaction_hash,
    reactants_to_fingerprint,
)
from aizynthfinder.utils.models import CUSTOM_OBJECTS, load_keras_model


In [3]:
data_reactions = pd.read_json("C:/Users/Yassine/Desktop/aizynth/uspto-reactions.json")
data_templates = pd.read_json("C:/Users/Yassine/Desktop/aizynth/uspto-templates.json")

In [4]:
data_templates_limited = data_templates.iloc[:100]
data_reactions_limited = data_reactions.iloc[:100]

In [5]:
data_templates_limited

,reaction_id,reactants,intra_only,products,reaction_smarts,dimer_only,necessary_reagent
0,0,Cl-[S;H0;D4;+0:1](-[C:2])(=[O;D1;H0:3])=[O;D1;...,0.0,[C:5]-[O;H0;D2;+0:6]-[S;H0;D4;+0:1](-[C:2])(=[...,[C:5]-[O;H0;D2;+0:6]-[S;H0;D4;+0:1](-[C:2])(=[...,0.0,
1,1,Cl-[S;H0;D4;+0:1](-[C;D1;H3:2])(=[O;D1;H0:3])=...,0.0,[C:5]-[O;H0;D2;+0:6]-[S;H0;D4;+0:1](-[C;D1;H3:...,[C:5]-[O;H0;D2;+0:6]-[S;H0;D4;+0:1](-[C;D1;H3:...,0.0,
2,2,Cl-[S;H0;D4;+0:1](-[C:2])(=[O;D1;H0:3])=[O;D1;...,0.0,[C:2]-[S;H0;D4;+0:1](=[O;D1;H0:3])(=[O;D1;H0:4...,[C:2]-[S;H0;D4;+0:1](=[O;D1;H0:3])(=[O;D1;H0:4...,0.0,
3,3,Cl-[S;H0;D4;+0:1](-[C:2])(=[O;D1;H0:3])=[O;D1;...,0.0,[C:5]-[O;H0;D2;+0:6]-[S;H0;D4;+0:1](-[C:2])(=[...,[C:5]-[O;H0;D2;+0:6]-[S;H0;D4;+0:1](-[C:2])(=[...,0.0,
4,4,Cl-[S;H0;D4;+0:1](-[C;D1;H3:2])(=[O;D1;H0:3])=...,0.0,[C:5]-[O;H0;D2;+0:6]-[S;H0;D4;+0:1](-[C;D1;H3:...,[C:5]-[O;H0;D2;+0:6]-[S;H0;D4;+0:1](-[C;D1;H3:...,0.0,
...,...,...,...,...,...,...,...
95,95,O-[CH3;D1;+0:1].[O;D1;H0:2]=[C:3]-[OH;D1;+0:4],0.0,[CH3;D1;+0:1]-[O;H0;D2;+0:4]-[C:3]=[O;D1;H0:2],[CH3;D1;+0:1]-[O;H0;D2;+0:4]-[C:3]=[O;D1;H0:2]...,0.0,
96,96,C-[c;H0;D3;+0:1](:[#7;a;+:2]):[c:3]:[#7;a;+:4],1.0,[#7;a;+:2]:[cH;D2;+0:1]:[c:3]:[#7;a;+:4],[#7;a;+:2]:[cH;D2;+0:1]:[c:3]:[#7;a;+:4]>>C-[c...,0.0,
97,97,C-[CH2;D2;+0:1]-[C:2],1.0,[C:2]-[CH3;D1;+0:1],[C:2]-[CH3;D1;+0:1]>>C-[CH2;D2;+0:1]-[C:2],0.0,
98,98,Cl-c1:c:c:c:c(-C(=[O;H0;D1;+0:1])-O-O):c:1.[C:...,0.0,[C:2]-[S;H0;D3;+0:3](-[C;D1;H3:4])=[O;H0;D1;+0:1],[C:2]-[S;H0;D3;+0:3](-[C;D1;H3:4])=[O;H0;D1;+0...,0.0,


In [6]:
data_reactions_limited

,_id,reactants,products,spectators,source,source_id
0,0,[Br:1][CH2:2][CH2:3][OH:4].[CH2:5]([S:7](Cl)(=...,[CH2:5]([S:7]([O:4][CH2:3][CH2:2][Br:1])(=[O:9...,C(N(CC)CC)C,uspto,d55b478b903cc66713bcd03da0cb590206c063770b2566...
1,1,[Br:1][CH2:2][CH2:3][CH2:4][OH:5].[CH3:6][S:7]...,[CH3:6][S:7]([O:5][CH2:4][CH2:3][CH2:2][Br:1])...,C(N(CC)CC)C,uspto,4f4596769c26752550f51dfa2e2ed3c85314e993098a21...
2,2,[CH2:1]([Cl:4])[CH2:2][OH:3].CCOCC.[CH2:10]([S...,[CH2:10]([S:14]([O:3][CH2:2][CH2:1][Cl:4])(=[O...,C(N(CC)CC)C,uspto,b437d8086077a93b6c04154fe4ea6b3b8a718cf617fcfd...
3,3,[Br:1][CH2:2][CH2:3][OH:4].[CH2:5]([S:7](Cl)(=...,[CH2:5]([S:7]([O:4][CH2:3][CH2:2][Br:1])(=[O:9...,C(N(CC)CC)C,uspto,11b36b516a4e353d293245bc8a3af922dc42c0e5d1e06c...
4,4,[Br:1][CH2:2][CH2:3][CH2:4][OH:5].[CH3:6][S:7]...,[CH3:6][S:7]([O:5][CH2:4][CH2:3][CH2:2][Br:1])...,C(N(CC)CC)C,uspto,ef05638cb9f9190bf222b9a92a7159e96ee090c12cf110...
...,...,...,...,...,...,...
95,95,[CH3:1][C:2]1[CH:21]=[CH:20][C:5]2[O:6][CH:7](...,[CH3:1][C:2]1[CH:21]=[CH:20][C:5]2[O:6][CH:7](...,,uspto,a888f8349b8e2090ad849fc83992ec014cbe8dd866835b...
96,96,CC1C(CSC)=[N+]([O-])C2C(=CC=CC=2)[N+]=1[O-].C[...,[CH3:31][S:29]([CH2:28][C:27]1[CH:18]=[N+:19](...,,uspto,e5eb326910ae3586b98dbb225489c01fba9135496e7c96...
97,97,C(SCC1C=[N+]([O-])C2C(=CC=CC=2)[N+]=1[O-])CC.[...,[CH2:18]([S:21]([CH2:23][C:24]1[CH:33]=[N+:32]...,,uspto,284a57e657a8469d48edcf166b75420be452de8b33826a...
98,98,ClC1C=CC=C(C(OO)=[O:9])C=1.[CH3:12][S:13][CH2:...,[CH3:12][S:13]([CH2:14][C:15]1[C:24]([C:25]([O...,C(Cl)(Cl)Cl,uspto,eea42804efc3e323c2b3c6ef130469926fcc00b667af56...


In [7]:
def get_random_row(lib):
    r = random.randint(0,lib.shape[0]-1)
    return [lib.iloc[r]]

In [8]:
def random_sample_lib(library, lib2):
    to_return = []
    for val, row in lib2.iterrows():
        if (type(row.reactants) == str):
            mols = create_reactants_molecules(row.reactants)
        else:
            continue
        try:
            ref_mol = Molecule(smiles=row.products, sanitize=True)
        except MoleculeException:
            continue
            
        new_product = None
        for template_row in get_random_row(library): #SAMPLER 
            #if row.template_hash == template_row.template_hash: #Check
            #    continue
            #smarts_fwd = reverse_template(template_row.retro_template) #Check
            if (type(template_row["reaction_smarts"]) != str):
                continue
            smarts_fwd = reverse_template(template_row["reaction_smarts"])
            try:
                new_product = Reaction(mols=mols, smarts=smarts_fwd).apply()[0][0]
            except (ValueError, IndexError):
                continue
            if new_product.basic_compare(ref_mol):
                continue
            break  # If we have reached here, we have found a match that fits all criteria

        if not new_product:
            continue
        
        to_return += [val,new_product]
    return to_return

In [9]:
def _sample_lib(library, lib2):
    to_return = []
    for val, row in lib2.iterrows():
        if (type(row.reactants) == str):
            mols = create_reactants_molecules(row.reactants)
        else:
            continue
        try:
            ref_mol = Molecule(smiles=row.products, sanitize=True)
        except MoleculeException:
            continue
            
        new_product = None
        for idx,template_row in library.iterrows(): #SAMPLER 
            #if row.template_hash == template_row.template_hash: #Check
            #    continue
            #smarts_fwd = reverse_template(template_row.retro_template) #Check
            if (idx != 15):
                continue
            if (type(template_row["reaction_smarts"]) != str):
                continue
            smarts_fwd = reverse_template(template_row["reaction_smarts"])
            try:
                new_product = Reaction(mols=mols, smarts=smarts_fwd).apply()[0][0]
            except (ValueError, IndexError):
                continue
            if new_product.basic_compare(ref_mol):
                continue
            break  # If we have reached here, we have found a match that fits all criteria

        if not new_product:
            continue
        
        to_return += [val,new_product]
    return to_return

In [36]:
y = random_sample_lib(data_templates, data_reactions)

In [37]:
z = y[1::2]

In [38]:
n = len(z)
arr_to_dat = np.empty(n, dtype="object")
for x in range(n):
    arr_to_dat[x] = data_reactions.iloc[y[2*x]].reactants + ">>" + z[x].smiles
arr_to_dat

array(['Cl.[OH:2][C:3]1[CH:15]=[CH:14][CH:13]=[CH:12][C:4]=1[C:5]([CH2:7][CH2:8][C:9]([OH:11])=[O:10])=[O:6].[CH3:16]O>>C(C[CH2:8][C:9](=[O:10])[OH:11])c1[c:3]([OH:2])[cH:15][cH:14][cH:13][cH:12]1',
       '[CH3:1][O:2][C:3](=[O:16])[C:4]1[CH:9]=[C:8]([O:10][CH3:11])[C:7]([C:12]#[N:13])=[C:6]([O:14][CH3:15])[CH:5]=1.Cl.[H][H]>>OCc1[cH:5][c:6]([O:14][CH3:15])[c:7]([C:12]#[N:13])[c:8]([O:10][CH3:11])[cH:9]1',
       '[O:1]1[CH:5]=[CH:4][C:3]([C:6]([O:8]CC)=O)=[CH:2]1.[NH2:11][C:12]1[CH:26]=[CH:25][CH:24]=[CH:23][C:13]=1[CH2:14][CH2:15][CH:16]1[CH2:21][CH2:20][CH2:19][CH2:18][N:17]1[CH3:22]>>O=C(O)[c:3]1[cH:2][o:1][cH:5][cH:4]1',
       ...,
       '[F:1][C:2]([F:25])([F:24])[C:3]1[CH:4]=[CH:5][C:6]([O:9][C@H:10]2[C@@H:15]3[CH2:16][C@@H:12]([CH2:13][N:14]3C(OC(C)(C)C)=O)[CH2:11]2)=[N:7][CH:8]=1.Cl>>C1N[C@@H]2[C@H:10]([O:9][c:6]3[cH:5][cH:4][c:3]([C:2]([F:1])([F:24])[F:25])[cH:8][n:7]3)[CH2:11][C@H:12]1[CH2:16]2',
       '[C:1]([C:3]1[C:4]([C:22]2[CH:27]=[CH:26][C:25]([O:28][C:29]3[CH:34]=

In [12]:
import numpy as np

In [39]:
false_reactions = pd.DataFrame(data=arr_to_dat)

In [40]:
false_reactions.to_csv("false_reactions_all_reactions_16.csv")

In [ ]:
reac_has = reaction_hash(data_reactions_limited.iloc[15].reactants, y[1])

In [ ]:
reac_has

In [ ]:
def random_application(library: pd.DataFrame, config: Config, _) -> _DfGenerator:
    random.seed(100)  # To have reproducible results

    def random_sampler(_):
        for _ in range(config["negative_data"]["random_trials"]):
            template_index = random.randrange(len(library))
            yield library.iloc[template_index]

    yield from _sample_library(library, config, random_sampler)